In [ ]:
Following are the steps to install the package on SageMaker notebook :

STEP 1: run !which python in your sell and get the location of your python packages. For example, in my case it is : /home/ec2-user/anaconda3/envs/python3/bin/python

STEP 2: run !/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade  --ignore-installed --force-reinstall https://s3-us-west-2.amazonaws.com/move-dl-common-binary-distrubution/python/move_dl_common_api-3.2.131-release.tar.gz

STEP 3: End (hopefully you installed it successfully!)

In [ ]:
!/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade --ignore-installed --force-reinstall https://s3-us-west-2.amazonaws.com/move-dl-common-binary-distrubution/python/move_dl_common_api-3.2.131-release.tar.gz

In [1]:
!pip install pyarrow
!pip install dask
from move_dl_common_api.athena_util import AthenaUtil
from datetime import datetime
from pytz import timezone
from datetime import datetime, timedelta
import json
import logging
import sys
import os
import warnings
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pickle
from scipy import stats
import pyarrow.parquet as pq
import dask.dataframe as dd
import s3fs
import multiprocessing
import seaborn as sns
from matplotlib import pyplot
s3 = s3fs.S3FileSystem()
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

logging.basicConfig()

In [2]:
class AthenaUtilities(object):
    __s3_path = None
    __db_name = None
    __util = None
    __log = None
    
    def __init__(cls, db_name, db_path_on_s3, logger_name):
        cls.__log = FeatureEngineeringUtilities.get_logger(logger_name)
        cls.__db_name = db_name
        cls.__s3_path = db_path_on_s3
        cls.__util = AthenaUtil(s3_staging_folder = cls.__s3_path)
        
    def set_s3_path(cls, path):
        cls.__s3_path = path
        cls.__util = AthenaUtil(s3_staging_folder = cls.__s3_path)
    
    def set_db(cls, db):
        cls.__db_name = db
    
    def set_logger(cls, logger_name):
        cls.__log = FeatureEngineeringUtilities.get_logger(logger_name)
        
    def create_db(cls, db_name):
        """
            Description: Creat table
            @type  db_name : string
            @param db_name : DB Name
        """
        ctas_sql_create_db = """ CREATE DATABASE {} ;""".format(db_name)
        res = cls.__util.execute_query(sql_query = ctas_sql_create_db)
            
    @staticmethod  
    def get_logger(name, level=logging.INFO ):
        logger = logging.getLogger(name)
        logger.setLevel(level)
        if logger.handlers:
            # or else, as I found out, we keep adding handlers and duplicate messages
            pass
        else:
            ch = logging.StreamHandler(sys.stderr)
            ch.setLevel(level)
            formatter = logging.Formatter('%(asctime)s %(name)s %(funcName)s pid = %(process)d line # : %(lineno)d %(levelname)s %(message)s')
            ch.setFormatter(formatter)
            logger.addHandler(ch)
        return logger
    
    def __is_successful_opr(cls, response):
        if int(response['ResponseMetadata']['HTTPStatusCode'])==200 :
            return 0
        else:
            return -1
    
    #Basic Feature Engineering Operations on Tables 
    def run_ctas_query(cls, ctas_query):
        res = cls.__util.execute_query(sql_query = ctas_query)
        
    def get_table_size(cls, table_name):
        """
            Description: Return back the number of rows of the table
            @type  table_name : string
            @param table_name : Table Name
            @rtype int : size of the table
        """
        try:
            ctas_sql_table_size = """ SELECT COUNT(*) from {}.{} ;""".format(cls.__db_name, table_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_table_size)
            nu_of_elems = res['ResultSet']['Rows'][0]['Data'][0]['VarCharValue']
            return int(nu_of_elems)
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = get_table_size - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return -1
        
    def get_table_names_in_db(cls):     
        try:
            ctas_sql_table_size = """ SHOW TABLES IN {};""".format(cls.__db_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_table_size)
            return res
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = get_table_names_in_db - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return -1
        
    def create_table_preview(cls, from_table_name, to_table_name, number_of_rows, extr_loc_by_table_name = False):
        """
            Description: Preview of the table 
            @type  from_table_name : string
            @param from_table_name : Table to make copy of
            @type  to_table_name : string
            @param to_table_name : preview table
            @rtype res : int
            @param res : 0 if succesfull else 1
        """
        try:
            if extr_loc_by_table_name is True:
                ctas_sql_preview_table = """CREATE TABLE {}.{}
                WITH (
                  external_location = '{}/tables/{}',
                  format='PARQUET'
                ) AS
                SELECT * FROM {}.{} limit {};""".format(
                           cls.__db_name, 
                           to_table_name,
                           cls.__s3_path,
                           to_table_name,
                           cls.__db_name, 
                           from_table_name, str(number_of_rows) )
            else:
                ctas_sql_preview_table = """CREATE TABLE {}.{}
                WITH (
                  format='PARQUET'
                ) AS
                SELECT * FROM {}.{} limit {};""".format(
                           cls.__db_name, 
                           to_table_name,
                           cls.__db_name, 
                           from_table_name, str(number_of_rows) )
                    
            res = cls.__util.execute_query(sql_query = ctas_sql_preview_table)
            return cls.__is_successful_opr(res)
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = copy_table - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))

        
    def copy_table(cls, table_name, copy_table_name, extr_loc_by_table_name = False):
        """
            Description: Make a copy of a table 
            @type  table_name : string
            @param table_name : Table to make copy of
            @type  copy_table_name : string
            @param copy_table_name : Copy table
            @rtype res : int
            @param res : 0 if succesfull else 1
        """
        try:
            if extr_loc_by_table_name is True:
                ctas_sql_copy_table = """CREATE TABLE {}.{}
                WITH (
                  external_location = '{}/tables/{}',
                  format='PARQUET'
                ) AS
                SELECT * FROM {}.{};""".format(
                           cls.__db_name, 
                           copy_table_name,
                           cls.__s3_path,
                           to_table_name,
                           cls.__db_name, 
                           table_name)
            else:
                ctas_sql_copy_table = """CREATE TABLE {}.{}
                WITH (
                  format='PARQUET'
                ) AS
                SELECT * FROM {}.{};""".format(
                           cls.__db_name, 
                           copy_table_name,
                           cls.__db_name, 
                           table_name)
                
            res = cls.__util.execute_query(sql_query = ctas_sql_copy_table)
            return cls.__is_successful_opr(res)
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = copy_table - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
               
    def drop_table(cls, table_name):
        """
            Description: Drop table 
            @type  table_name : string
            @param table_name : Table Name 
            @rtype res : int
            @param res : 0 if succesfull else 1
        """
        try:
            ctas_sql_drop_table = """ DROP TABLE {}.{} ;""".format(cls.__db_name, table_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_drop_table)
            return cls.__is_successful_opr(res)
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = drop_table - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return -1
    
    def repair_partioned_table(cls, table_name):
        """
            Description: Repair partioned table
            @type  table_name : string
            @param table_name : Table Name 
            @rtype res : int
            @param res : 0 if succesfull else 1
        """
        try:
            ctas_sql_repair_table = """ MSCK REPAIR TABLE {}.{} ;""".format(cls.__db_name, table_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_repair_table)
            return cls.__is_successful_opr(res)
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = repair_partioned_table - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return -1
    
    def random_sample_based_on_string_attr(cls, to_table_name, from_table_name, 
                                           string_atr, sample_rate = 20, extr_loc_by_table_name = False):
        """
            Description: Random sample a table based on string attributes
            @type  to_table_name : string
            @param to_table_name : Final table
            @type  from_table_name : string
            @param from_table_name : table that will be sampled
            @type  string_atr : string
            @param string_atr : string attribute that will be hashed for sampling
            @type  sample_rate : int
            @param sample_rate : rate of sampling for example 20 means final table size should be around 1/20 of original
            @rtype res : int
            @param res : 0 if succesfull else 1
        """
        try:
            if extr_loc_by_table_name is True:
                ctas_sql_random_sampling = """CREATE TABLE {}.{}
                WITH (
                  external_location = '{}/tables/{}',
                  format='PARQUET'
                ) AS
                SELECT * 
                FROM {}.{} 
                WHERE MOD(from_big_endian_64(xxhash64(to_utf8({}))), CAST({} AS BIGINT)) = 0;
                """.format(cls.__db_name, 
                           to_table_name,
                           cls.__s3_path,
                           to_table_name,
                           cls.__db_name, 
                           from_table_name, 
                           string_atr, 
                           str(sample_rate))
            else:
                ctas_sql_random_sampling = """CREATE TABLE {}.{}
                WITH (
                  format='PARQUET'
                ) AS
                SELECT * 
                FROM {}.{} 
                WHERE MOD(from_big_endian_64(xxhash64(to_utf8({}))), CAST({} AS BIGINT)) = 1;
                """.format(cls.__db_name, 
                           to_table_name,
                           cls.__db_name, 
                           from_table_name, 
                           string_atr, 
                           str(sample_rate))

            res = cls.__util.execute_query(sql_query = ctas_sql_random_sampling)
            return cls.__is_successful_opr(res)
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = random_sample_based_on_string_attr - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return -1;
    
    def creat_table_in_time_window(cls, from_table_name, to_table_name, time_attr_of_from_table, 
                                   start_time, end_time, extr_loc_by_table_name = False):
        """
            Description: Filter elements in the time frame [start_time, end_time] 
            @type  from_table_name : string
            @param from_table_name : Table in which we want to get time frame of elements 
            @type  to_table_name : string
            @param to_table_name : Final that contans time_filtered
            @type  time_attr_of_from_table : string
            @param time_attr_of_from_table : time attribute of the from table to put time fileter
            @type  start_time : string
            @param start_time : start time in mst format
            @type  end_time : string
            @param end_time : end time in mst format
            @rtype res : int
            @param res : 0 if succesfull else 1
            
        """
        try:
            if extr_loc_by_table_name is True:
                ctas_sql_time_window = """CREATE TABLE {}.{}
                WITH (
                  external_location = '{}/tables/{}',
                  format='PARQUET'
                ) AS
                SELECT * 
                FROM {}.{} 
                WHERE DATE({}) BETWEEN DATE('{}') AND DATE('{}');
                """.format(cls.__db_name, 
                           to_table_name,
                           cls.__s3_path,
                           to_table_name,
                           cls.__db_name, 
                           from_table_name, 
                           time_attr_of_from_table, 
                           start_time,
                           end_time
                          )
            else:
                ctas_sql_time_window = """CREATE TABLE {}.{}
                WITH (
                  format='PARQUET'
                ) AS
                SELECT * 
                FROM {}.{} 
                WHERE DATE({}) BETWEEN DATE('{}') AND DATE('{}');
                """.format(cls.__db_name, 
                           to_table_name,
                           cls.__db_name, 
                           from_table_name, 
                           time_attr_of_from_table, 
                           start_time,
                           end_time
                          )
                
            res = cls.__util.execute_query(sql_query = ctas_sql_time_window)
            return cls.__is_successful_opr(res)
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = creat_table_in_time_window - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return -1
    
    def remove_null_columns(cls, from_table_name, to_table_name, column_list, 
                            extr_loc_by_table_name = False):
        """
            Description: Filter null elements related to column_list 
            @type  from_table_name : string
            @param from_table_name : Table in which we want to get time frame of elements
            @type  to_table_name : string
            @param to_table_name : Final that contans time_filtered
            @type  column_list : list of strings
            @param column_list : columns in which we want to remove null values
            @rtype res : int
            @param res : 0 if succesfull else 1     
        """
        try:
            if column_list is None:
                return 0
            else:
                length = len(column_list)
                list_counter = 0
                if extr_loc_by_table_name is True:
                    ctas_sql_remove_all_nulls = """CREATE TABLE {}.{}
                                WITH (
                                external_location = '{}/tables/{}',
                                format='PARQUET'
                                ) AS
                                SELECT * 
                                FROM {}.{} 
                                WHERE {} IS NOT NULL {};
                                """.format( cls.__db_name, 
                                           to_table_name,
                                           cls.__s3_path,
                                           to_table_name,
                                           cls.__db_name, 
                                           from_table_name, 
                                           '{}','{}') 
                else:
                    ctas_sql_remove_all_nulls = """CREATE TABLE {}.{}
                                WITH (
                                format='PARQUET'
                                ) AS
                                SELECT * 
                                FROM {}.{} 
                                WHERE {} IS NOT NULL {};
                                """.format( cls.__db_name, 
                                           to_table_name,
                                           cls.__db_name, 
                                           from_table_name, 
                                           '{}','{}')          
                for column in column_list:
                    # Last element
                    if length == list_counter + 1:
                        ctas_sql_remove_all_nulls = ctas_sql_remove_all_nulls.format(column,'')
                    else:
                        ctas_sql_remove_all_nulls = ctas_sql_remove_all_nulls.format(column,'AND {} IS NOT NULL')
                    list_counter = list_counter + 1

            res = cls.__util.execute_query(sql_query = ctas_sql_remove_all_nulls)
            return cls.__is_successful_opr(res)
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = remove_null_columns - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return -1
        
    def select_columns_from_table(cls, from_table_name, to_table_name, column_list, 
                            extr_loc_by_table_name = False):
        """
            Description: Select the listed columns
            @type  from_table_name : string
            @param from_table_name : Table in which we want to get time frame of elements
            @type  to_table_name : string
            @param to_table_name : Final that contans time_filtered
            @type  column_list : list of strings
            @param column_list : columns in which we want to remove null values
            @rtype res : int
            @param res : 0 if succesfull else 1     
        """
        try:
            if column_list is None:
                return 0
            else:
                length = len(column_list)
                list_counter = 0
                arg = ''
                for column in column_list:
                    if list_counter == length - 1 :
                        arg = arg + cls.__db_name+'.' + from_table_name + '.' + column
                    else:
                        arg = arg + cls.__db_name+'.' + from_table_name + '.' + column + ','
                    list_counter = list_counter + 1
                if extr_loc_by_table_name is True:
                    ctas_sql_remove_all_nulls = """CREATE TABLE {}.{}
                                WITH (
                                external_location = '{}/tables/{}',
                                format='PARQUET'
                                ) AS
                                SELECT {} 
                                FROM {}.{};
                                """.format(cls.__db_name, 
                                           to_table_name, 
                                           cls.__s3_path,
                                           to_table_name,
                                           arg,
                                           cls.__db_name, 
                                           from_table_name) 
                else:
                    ctas_sql_remove_all_nulls = """CREATE TABLE {}.{}
                                WITH (
                                format='PARQUET'
                                ) AS
                                SELECT {} 
                                FROM {}.{};
                                """.format(cls.__db_name, 
                                           to_table_name, 
                                           arg,
                                           cls.__db_name, 
                                           from_table_name)          

            res = cls.__util.execute_query(sql_query = ctas_sql_remove_all_nulls)
            return cls.__is_successful_opr(res)
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = select_columns_list_from_table - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return -1
    
    #Basic Statistical Operation on Athena Tables
    def get_min_of_an_attribute(cls, table_name, attr_name):
        """
            Description: Get min value of numerical value from Athena Table
            @type  table_name : string
            @param table_name : Table in which we want min value
            @type  attr_name : string
            @param attr_name : name of attribute
            @rtype res : string
            @param res : min value
            
        """
        try:
            ctas_sql_get_min =""" 
            SELECT MIN({}) FROM {}.{};
            """.format(attr_name, cls.__db_name, table_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_get_min)
            return res['ResultSet']['Rows'][0]['Data'][0]['VarCharValue']
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = get_min_of_an_attribute - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return None
    
    def get_max_of_an_attribute(cls, table_name, attr_name):
        """
            Description: Get max value of numerical value from Athena Table 
            @type  table_name : string
            @param table_name : Table in which we want min value
            @type  attr_name : string
            @param attr_name : name of attribute
            @rtype res : string
            @param res : max value
            
        """
        try:
            ctas_sql_get_max =""" 
            SELECT MAX({}) FROM {}.{};
            """.format(attr_name, cls.__db_name, table_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_get_max)
            return res['ResultSet']['Rows'][0]['Data'][0]['VarCharValue']
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = get_max_of_an_attribute - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return None
    
    def get_avg_of_an_attribute(cls, table_name, attr_name):
        """
            Description: Get average value of numerical value from Athena Table
            @type  table_name : string
            @param table_name : Table in which we want min value
            @type  attr_name : string
            @param attr_name : name of attribute
            @rtype res : string
            @param res : avg of an attribute
        """
        try:
            ctas_sql_get_avg =""" 
            SELECT AVG(CAST({} AS DECIMAL)) FROM {}.{};
            """.format(attr_name, cls.__db_name, table_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_get_avg)
            return res['ResultSet']['Rows'][0]['Data'][0]['VarCharValue']
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = get_avg_of_an_attribute - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return None
    
    def get_std_of_an_attribute(cls, table_name, attr_name):
        """
            Description: Get std value of numerical value from Athena Table
            @type  table_name : string
            @param table_name : Table in which we want min value 
            @type  attr_name : string
            @param attr_name : name of attribute
            @rtype res : string
            @param res : std of data  
        """
        try:
            ctas_sql_get_std = """ SELECT STDDEV(CAST({} AS DECIMAL)) FROM {}.{};""".format(
                   attr_name, 
                   cls.__db_name, 
                   table_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_get_std)
            return res['ResultSet']['Rows'][0]['Data'][0]['VarCharValue']
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = get_std_of_an_attribute - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return None
    
    def get_percentile_of_an_attribute(cls, table_name, attr_name, p):
        """
            Description: Get percentile p value of numerical value from Athena Table
            
            @type  table_name : string
            @param table_name : Table in which we want min value
            
            @type  p : float
            @param p : percentile should be between [0,1]
            
            @rtype res : string
            @param res : result of the data
            
        """
        try:
            ctas_sql_get_percentile =""" SELECT APPROX_PERCENTILE(CAST({} AS DECIMAL),{}) FROM {}.{};""".format(
                   attr_name, 
                   str(p), 
                   cls.__db_name, 
                   table_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_get_percentile)
            return res['ResultSet']['Rows'][0]['Data'][0]['VarCharValue']
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = get_percentile_of_an_attribute - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return None
        
    def get_histogram_of_an_attribute(cls, table_name, bucket, attr_name):
        """
            Description: Get histogram of numerical value from Athena Table 
            @type  table_name : string
            @param table_name : Table in which we want min value
            @type  bucket : int
            @param bucket : number of buckets
            @type  attr_name : string
            @param attr_name : name of attribute
            @rtype res : dict
            @param res : histogram
            
        """
        try:
            ctas_sql_get_histogram =""" SELECT NUMERIC_HISTOGRAM({}, (CAST({} AS DECIMAL))) FROM {}.{};""".format(
                   str(bucket),
                   attr_name, 
                   cls.__db_name, 
                   table_name)
            res = cls.__util.execute_query(sql_query = ctas_sql_get_histogram)
            
            def parse_res(res):
                r = list(res.replace("{","").replace("}","").replace(" ","").split(','))
                final = dict()
                for item in r:
                    tmp = list(item.split('='))
                    final[float(tmp[0])]=float(tmp[1])
                return final      
            
            return parse_res(res['ResultSet']['Rows'][0]['Data'][0]['VarCharValue'])
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = get_histogram_of_an_attribute - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return None 

    def get_cor_of_attributes(cls, table_name, attr_1, attr_2):
        """
            Description: Get covarriance of attr_1 and attr_2 
            @type  table_name : string
            @param table_name : Table in which we want min value
            @type  attr_1 : int
            @param attr_1 : number of buckets
            @type  attr_2 : int
            @param attr_2 : number of buckets
            @rtype res : float
            @param res : covvar     
        """
        try:
            corr =""" SELECT corr((CAST({} AS DECIMAL)), (CAST({} AS DECIMAL))) FROM {}.{};""".format(
                   attr_1,
                   attr_2, 
                   cls.__db_name, 
                   table_name)
            res = cls.__util.execute_query(sql_query = corr) 
            return float(res['ResultSet']['Rows'][0]['Data'][0]['VarCharValue'])
       
        except Exception as message:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            cls.__log.error("function = get_cor_of_attributes - exception in line_number = %s - hint = %s " % (
                exc_tb.tb_lineno, message.__str__()))
            return None    